In [ ]:
# 目标 让 llm 具有调用工具的能力，也就是所谓的 agent

In [1]:
# 安装依赖
%uv add -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

Note: you may need to restart the kernel to use updated packages.


Resolved 156 packages in 2.20s
Prepared 14 packages in 2.16s
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 14 packages in 653ms
 + aiosqlite==0.21.0
 + anthropic==0.49.0
 + distro==1.9.0
 + jiter==0.9.0
 + langchain-anthropic==0.3.10
 + langgraph==0.3.18
 + langgraph-checkpoint==2.0.21
 + langgraph-checkpoint-sqlite==2.0.6
 + langgraph-prebuilt==0.1.4
 + langgraph-sdk==0.1.58
 + msgpack==1.1.0
 + regex==2024.11.6
 + tavily-python==0.5.1
 + tiktoken==0.9.0


In [ ]:
# 相关环境变量设置
import sys 
sys.path.append("..") 
from config import config_loader

config_loader.load_env()

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)

# 测试工具是否可以正常使用
search_results = search.invoke("北京未来一周的天气情况")
print(search_results)

# 工具可以都放到一个列表中 给模型调用
tools = [search]

[{'title': '【北京天气预报】北京天气预报一周,15天,30天天气查询-2345天气 ...', 'url': 'http://waptianqi.2345.com/', 'content': '【北京天气预报】北京天气预报一周,15天,30天天气查询-2345天气预报天气预报一周,7天,10天,15天,未来一周天气预报查询_2345天气预报\n\n\n2345天气王\n2345天气王\n\n\n本地天气\n国际天气\n卫星云图\n\n北京天气 ==== [切换]\n\n\n多云\n-11\xa0~\xa0-1℃\n\n西北风 : 3级\n湿度 : 21%\n气压 : 1036 hPa\n\n7日天气预报 放到桌面\n\n今天_02/08_ 晴转多云 -11~-1℃\n明天_02/09_ 晴 -10~1℃\n\n周一_02/10_\n晴转多云\n-6~4℃\n\n\n周二_02/11_\n多云\n-5~5℃\n\n\n周三_02/12_\n晴\n-4~6℃\n\n\n周四_02/13_\n晴\n-3~7℃\n\n\n周五_02/14_\n多云\n-2~8℃\n\n\n查看15日天气\n（免费下载天气王APP）', 'score': 0.79022324}, {'title': '中国气象局-天气预报- 北京', 'url': 'https://weather.cma.cn/web/weather/54511.html', 'content': '主站首页\n领导主站\n部门概况\n新闻资讯\n信息公开\n服务办事\n天气预报\n首页\n天气实况\n气象公报\n气象预警\n城市预报\n天气资讯\n气象专题\n气象科普\n 首页 国内 北京 北京\n国内 \n|\n北京 \n|\n北京 \n更新\n\xa0\n \xa0  \xa0  \xa0  \xa0\n7天天气预报（2025/02/08 08:00发布）\n星期六\n02/08\n晴\n北风\n微风\n-1℃\n-10℃\n多云\n北风\n微风\n星期日\n02/09\n晴\n南风\n微风\n1℃\n-10℃\n晴\n北风\n微风\n星期一\n02/10\n晴\n南风\n微风\n4℃\n-5℃\n多云\n北风\n微风\n星期二\n02/11\n多云\n北风\n微风\n5℃\n-4℃\n多云\n西北风\n

In [4]:
# 让 llm 使用工具
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain_core.messages.human import HumanMessage

# 定义大模型
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# 给该大模型绑定工具
llm_with_tools = llm.bind_tools(tools)

# 测试普通消息
response = llm_with_tools.invoke([HumanMessage(content='你好呀！')])
print(f'普通消息调用结果：{response.content}')
print(f"工具调用信息：{response.tool_calls}", )

普通消息调用结果：你好！有什么我可以帮你的吗？
工具调用信息：[]


In [ ]:
response = llm_with_tools.invoke("北京未来一周天气")
# 可以看到这个是空的
print(f"调用工具消息结果：{response.content}" )
# 这个不是调用工具的结果，而是大模型告诉我们要调用这个工具 所以需要创建代理
print(f"工具调用信息：{response.tool_calls}")

调用工具消息结果：
工具调用信息：[{'name': 'tavily_search_results_json', 'args': {'query': '北京未来一周天气'}, 'id': '5d7d9d40-c529-4e84-8ded-8a0a9c7505a5', 'type': 'tool_call'}]


In [6]:
from langgraph.prebuilt import create_react_agent

# 创建 agent
agent_executor = create_react_agent(llm, tools)

# 看看不需要工具时候的响应
response = agent_executor.invoke({"messages": [HumanMessage(content="你好~")]})

response["messages"]

[HumanMessage(content='你好~', additional_kwargs={}, response_metadata={}, id='825f118c-55f2-49de-9511-46be978f46cd'),
 AIMessage(content='你好！有什么我可以帮你的吗？', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-c18dc2d4-e6f0-426e-9a4c-1d11253e4642-0', usage_metadata={'input_tokens': 49, 'output_tokens': 9, 'total_tokens': 58, 'input_token_details': {'cache_read': 0}})]

In [7]:
# 测试工具调用
response = agent_executor.invoke({"messages": [HumanMessage(content="北京今天天气情况")]})

print(type(response))
response["messages"]

<class 'langgraph.pregel.io.AddableValuesDict'>


[HumanMessage(content='北京今天天气情况', additional_kwargs={}, response_metadata={}, id='775411f0-d280-4186-8742-413f07f61b5e'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "\\u5317\\u4eac\\u4eca\\u5929\\u5929\\u6c14"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-14edb119-55a4-4124-93d9-c59c1cc8ac93-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '北京今天天气'}, 'id': 'ff01573c-7e20-4dc2-8e2e-028ce6aef2c5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 51, 'output_tokens': 12, 'total_tokens': 63, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='[{"title": "【北京天气预报】北京天气预报7天,10天,15天_全国天气网", "url": "https://tianqi.so.com/weather/", "content": "全国天气网\\n首页\\n国内天气\\n空气质量\\n国际天气\\n景点天气\\n天气新闻\\n专业天气\\n收藏\\n北京[切换]当前时间：2025-02-08周六09:42\\n-7\\n晴\\n空气 优西北风 5级\\n今天 (02-08)\\n晴转多云\\n-11/-

In [8]:
# 流式传输 显式实时进度
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="今天北京天气怎么样？")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

今天北京天气怎么样？
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (1ac5eef3-6ba5-4129-85a4-a763f2664b67)
 Call ID: 1ac5eef3-6ba5-4129-85a4-a763f2664b67
  Args:
    query: 北京今天天气
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "【北京天气预报】北京天气预报7天,10天,15天_全国天气网", "url": "https://tianqi.so.com/weather/", "content": "全国天气网\n首页\n国内天气\n空气质量\n国际天气\n景点天气\n天气新闻\n专业天气\n收藏\n北京[切换]当前时间：2025-02-08周六09:42\n-7\n晴\n空气 优西北风 5级\n今天 (02-08)\n晴转多云\n-11/-1℃\n北风 微风\n明天 (02-09)\n晴\n-10/1℃\n南风 微风\n周一 (02-10)\n晴转多云\n-6/4℃\n东北风 微风\n周二 (02-11)\n多云\n-5/5℃\n北风 微风\n周三 (02-12)\n晴\n-4/6℃\n西北风 3-5级\n周四 (02-13)\n晴\n-3/7℃\n西南风 微风\n周五 (02-14)\n多云\n-2/8℃\n东北风 微风\n周六 (02-15)\n晴\n0/9℃\n西南风 微风\n周日 (02-16)\n晴\n0/10℃\n南风 微风\n周一 (02-17)\n多云转阴\n-1/8℃\n西南风 微风\n周二 (02-18)\n阴转多云\n-1/4℃\n南风 微风\n周三 (02-19)\n晴\n-1

In [9]:
# langchain-core>=0.3.37 的话 也可以添加 stream_mode="messages"
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="北京今天天气怎么样")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

今天|（2月26日）：晴，-1°~12°C，西南|风1级，空气质量良。请注意，此信息可能不完全|准确，建议您查看专业天气预报获取更详细和最新的信息。|

In [10]:
# agent 添加记忆
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
# 就是传参的时候 多一个 checkpointer 参数
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

# 设置对话线程
config = {"configurable": {"thread_id": "abc123"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="我的名字是张三")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='很高兴认识你，张三！', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-987779b5-f882-4781-9304-7560a3e7ee1c-0', usage_metadata={'input_tokens': 52, 'output_tokens': 9, 'total_tokens': 61, 'input_token_details': {'cache_read': 0}})]}}
----


In [11]:
# 测试记忆
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="我的名字是什么？")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='你的名字是张三。', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-54fea608-ffd0-466c-bb2e-7acd2d16d74e-0', usage_metadata={'input_tokens': 64, 'output_tokens': 7, 'total_tokens': 71, 'input_token_details': {'cache_read': 0}})]}}
----
